In [6]:
import numpy as np
import pandas as pd
import random #擬似乱数を生成するライブラリー 
import os
from PIL import Image, ImageOps #画像処理ライブラリー
from sklearn.metrics import accuracy_score #モデルの正解率を計算するライブラリー
import glob #ファイルパス名を取得するライブラリ
import matplotlib.pyplot as plt #図をプロットするためのライブラリー
%matplotlib inline #図をJupyter Notebook内に表示させる

UsageError: unrecognized arguments: #図をJupyter Notebook内に表示させる


In [ ]:
train_label_df = pd.read_csv('../data/raw/whale-categorization-playground/train.csv')

In [21]:
'''
## 人工的に画像データを増やす
train内にある約9800枚くらいの画像を適当に回したり、切り取りしたりして5倍に増やす
- 上下反転
- 左右反転
- 90度回転
- クリップ
'''

# train画像の読み込み(約9800枚くらい)
files = glob.glob('../data/raw/whale-categorization-playground/train/*')

# 出力先
output_dir = "../data/raw/whale-categorization-playground/train_augmented/"

# ランダムな領域の切り取りを行う関数
def random_crop(image, size=0.8):
    image = np.array(image, dtype=np.float64)

    height = image.shape[0]
    width = image.shape[1]
    crop_size = int(min(height, width) * size)

    top = np.random.randint(0, height - crop_size)
    left = np.random.randint(0, width - crop_size)
    bottom = top + crop_size
    right = left + crop_size
    image = image[top:bottom, left:right]

    return Image.fromarray(np.uint8(image))

for item in files:
    img = Image.open(item)
    file_name = os.path.basename(item)[:-4]
    img_size = img.size # 画像のサイズを取得

    img_flip = ImageOps.flip(img) # 画像の上下反転
    img_flip.save(output_dir + file_name + "_flip.jpg") # 画像保存

    img_mirror = ImageOps.mirror(img) # 画像の左右反転
    img_mirror.save(output_dir + file_name + "_mirror.jpg") # 画像保存

    img_rotate = img.rotate(90) # 画像の90度回転
    img_rotate.save(output_dir + file_name + "_rot90.jpg") # 画像保存

    img_crop = random_crop(img) # 画像の切り取り
    img_crop = img_crop.resize((256, 256)) # 元のサイズに戻す
    img_crop.save(output_dir + file_name + "_crop.jpg") # 画像保存

[[242. 240. 236. ... 252. 252. 252.]
 [200. 199. 196. ... 252. 252. 252.]
 [228. 227. 226. ... 250. 250. 250.]
 ...
 [255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]]
[[[198. 191. 185.]
  [198. 191. 185.]
  [198. 191. 185.]
  ...
  [194. 185. 188.]
  [194. 185. 188.]
  [194. 185. 188.]]

 [[198. 191. 185.]
  [198. 191. 185.]
  [198. 191. 185.]
  ...
  [194. 185. 188.]
  [194. 185. 188.]
  [194. 185. 188.]]

 [[196. 189. 183.]
  [196. 189. 183.]
  [196. 189. 183.]
  ...
  [194. 185. 188.]
  [194. 185. 188.]
  [194. 185. 188.]]

 ...

 [[176. 176. 176.]
  [176. 176. 176.]
  [176. 176. 176.]
  ...
  [182. 173. 176.]
  [182. 173. 176.]
  [182. 173. 176.]]

 [[176. 176. 176.]
  [176. 176. 176.]
  [176. 176. 176.]
  ...
  [182. 173. 176.]
  [182. 173. 176.]
  [182. 173. 176.]]

 [[176. 176. 176.]
  [176. 176. 176.]
  [176. 176. 176.]
  ...
  [182. 173. 176.]
  [182. 173. 176.]
  [182. 173. 176.]]]
[[[181. 181. 181.]
  [181. 181. 18

In [ ]:
"""
## 画像データの行列への読み込み
scikit-learnで扱うために、全画像データをまとめて1つの大きな行列の形に変換します
"""

# train + train_augmentedの枚数
DIR1 = '../data/raw/whale-categorization-playground/train'
DIR2 = '../data/raw/whale-categorization-playground/train_augmented'
num_trian = len([name for name in os.listdir(DIR1) if os.path.isfile(os.path.join(DIR, name))])
num_trian_augmented = len([name for name in os.listdir(DIR2) if os.path.isfile(os.path.join(DIR, name))])
num_of_data_total = num_train + num_train_autmented

N_col = 128*128*3 # 行列の列数
X_train = np.zeros((num_of_data_total, N_col)) # 学習データ格納のためのゼロ行列生成
y_train = np.zeros((num_of_data_total)) # 学習データに対するラベルを格納するためのゼロ行列生成

# train画像を行列に読み込む
path_list = glob.glob("./Dataset_AVNIR2/train/clear_augmented/*") 
i_count = 0

for item in path_list:
    im = Image.open(item).convert('RGB')
    img_resize =im.resize((128,128))  # 画像のサイズ変更
    im_array = np.ravel(np.asarray(img_resize))　# 画像を配列に変換
    im_regularized = im_array/255.　# 正規化
    X_train[i_count,:] =  im_regularized　
    y_train[i_count] = 0　# ラベル
    i_count += 1

# train_augmented画像を行列に読み込む
path_list = glob.glob("./Dataset_AVNIR2/train/clear_augmented/*") 
i_count = 0